In [31]:
import tensorflow as tf
tf.__version__

'2.19.0'

In [32]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

MLPClassifier 로 신경망 구성

In [34]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(50,30), activation="logistic", solver="adam",
                    max_iter=1000)
mlp.fit(X_train, y_train)
mlp.score(X_test, y_test)
pred = mlp.predict(X_test)

import pandas as pd
import numpy as np
pd.crosstab(pred, y_test) #교차분류표

col_0,0,1,2
row_0,,,
0,14,0,0
1,0,18,0
2,0,0,13


In [35]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #  "TensorFlow 2.x를 설치했지만, 지금은 잠시 1.x 버전처럼 동작해줘"

In [36]:
X = tf.placeholder(tf.float32, [None, 4]) # "여기에 나중에 데이터를 넣어줄 거야"
y = tf.placeholder(tf.float32, [None, 3])

In [37]:
# W 가중치, b 편향 - 1layer
W1 = tf.Variable(tf.random_normal([4, 50]))
b1 = tf.Variable(tf.zeros([50]))
h1 = tf.nn.sigmoid(tf.matmul(X, W1)+b1)

In [38]:
# 2layer - 30 node 로 구성
W2 = tf.Variable(tf.random_normal([50, 30]))
b2 = tf.Variable(tf.zeros([30]))
h2 = tf.nn.sigmoid(tf.matmul(h1, W2)+b2)

In [39]:
# 3layer - 출력층
W3 = tf.Variable(tf.random_normal([30, 3]))
b3 = tf.Variable(tf.zeros([3]))
h3 = tf.nn.softmax(tf.matmul(h2, W3)+b3)

In [40]:
cross_entropy = -tf.reduce_sum(y*tf.log(h3), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)
train = tf.train.AdamOptimizer().minimize(loss)

In [41]:
import pandas as pd

feed_train = {X: X_train, y: pd.get_dummies(y_train).values}

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
    sess.run(train, feed_dict=feed_train)

    if i%100 == 0:
        tr_loss = sess.run(loss, feed_dict=feed_train)
        print("Step %d, %.5f" % (i, tr_loss))

Step 0, 2.27725
Step 100, 0.49325
Step 200, 0.22743
Step 300, 0.12828
Step 400, 0.08239
Step 500, 0.06816
Step 600, 0.06146
Step 700, 0.05774
Step 800, 0.05471
Step 900, 0.05248


In [42]:
correct_prediction = tf.equal(tf.argmax(h3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
feed_test = {X: X_test, y: pd.get_dummies(y_test).values}
print(sess.run(accuracy, feed_dict=feed_test))

1.0


In [43]:
sess.close()

keras로 신경망 구성

In [44]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [45]:
from tensorflow.keras.models import Sequential
model = Sequential()
model

<Sequential name=sequential_1, built=False>

In [46]:
from tensorflow.keras.layers import Input, Dense
model.add(Input(shape=(4,)))
model.add(Dense(50, activation='sigmoid'))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))

In [47]:
model

<Sequential name=sequential_1, built=True>

In [48]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [49]:
model.fit(X_train, y_train, epochs=300, verbose=1)

Epoch 1/300


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [ ]:
model.evaluate(X_test, y_test)

파이토치로 구성하기

In [1]:
import torch
torch.__version__

'2.8.0+cu126'

In [2]:
if torch.cuda.is_available():
  torch.cuda.get_device_name(0)

In [3]:
#iris dataset load
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
y = iris.target
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [4]:
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [5]:
import torch.nn as nn
model = nn.Sequential(
    nn.Linear(4,50), nn.Sigmoid(),
    nn.Linear(50,30), nn.Sigmoid(),
    nn.Linear(30,3), nn.Softmax(dim=1),
)

In [6]:
import torch.optim as optim

#w의 초기값 설정
for layer in model:
  if isinstance(layer, nn.Linear):
    nn.init.xavier_uniform_(layer.weight)

#손실함수
crit = nn.CrossEntropyLoss()
#옵티마이저
opt = optim.Adam(model.parameters())

In [7]:
#학습설정
for epoch in range(300):
  outputs = model(X_train)
  loss = crit(outputs, y_train.long()) #손실값
  opt.zero_grad() #새로운 w
  loss.backward()
  opt.step()

In [8]:
# 모델평가
with torch.no_grad():
  test_outputs = model(X_test)
  _, predicted = torch.max(test_outputs, 1)
  accuracy = (predicted==y_test).sum().item()/y_test.size(0)
  print(loss.item(), accuracy)

0.6345515847206116 0.9777777777777777


In [9]:
# keras 함수형api 로 신경망 구성하기

In [10]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import concatenate, Activation

In [11]:
from typing_extensions import Concatenate
input = Input(shape=(4,))
dense1 = Dense(50, activation='relu')(input)
dense2 = Dense(80, activation='relu')(input)
dense3 = Dense(30, activation='relu')(input)
x = concatenate([dense1, dense2, dense3])
output = Dense(3, activation='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 50)        │        250 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 80)        │        400 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 30)        │        150 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │        483 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,283 (5.01 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
history = model.fit(X_train, y_train, epochs=300, verbose=1)
history

Epoch 1/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.3847 - loss: 1.2593
Epoch 2/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3559 - loss: 1.1900
Epoch 3/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3548 - loss: 1.0836
Epoch 4/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.4727 - loss: 0.9893
Epoch 5/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9240 - loss: 0.8904
Epoch 6/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7135 - loss: 0.8056 
Epoch 7/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7343 - loss: 0.7291 
Epoch 8/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7571 - loss: 0.6915
Epoch 9/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9452 - loss: 0.6548
Epoch 10/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9570 - loss: 0.6168 
Epoch 11/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7978 - loss: 0.5828
Epoch 12/300
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6958 -

In [14]:
loss, acc = model.evaluate(X_test, y_test)
loss,acc

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 1.0000 - loss: 0.0772 


(0.07158350944519043, 1.0)

In [15]:
import numpy as np
pred_proba = model.predict(X_test[:5])
pred_class = np.argmax(pred_proba, axis=1)
pred_class

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


array([0, 1, 1, 0, 2])

In [16]:

from sklearn.metrics import classification_report
y_pred = pred_proba = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        14
         1.0       1.00      1.00      1.00        18
         2.0       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [17]:
model.save('iris_model.keras')

In [18]:
from tensorflow.keras.models import load_model

model = load_model('/content/iris_model.keras')
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 50)        │        250 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 80)        │        400 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 30)        │        150 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dense[0][0],      │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 3)         │        483 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,851 (15.05 KB)

 Trainable params: 1,283 (5.01 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,568 (10.04 KB)

In [19]:
json_str = model.to_json()
json_str

'{"module": "keras.src.models.functional", "class_name": "Functional", "config": {"name": "functional", "trainable": true, "layers": [{"module": "keras.layers", "class_name": "InputLayer", "config": {"batch_shape": [null, 4], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_layer"}, "registered_name": null, "name": "input_layer", "inbound_nodes": []}, {"module": "keras.layers", "class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": {"module": "keras", "class_name": "DTypePolicy", "config": {"name": "float32"}, "registered_name": null}, "units": 50, "activation": "relu", "use_bias": true, "kernel_initializer": {"module": "keras.initializers", "class_name": "GlorotUniform", "config": {"seed": null}, "registered_name": null}, "bias_initializer": {"module": "keras.initializers", "class_name": "Zeros", "config": {}, "registered_name": null}, "kernel_regularizer": null, "bias_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, 

In [20]:
import json

json_dic = json.loads(json_str)
with open('iris_model.json', 'w') as f:
  json.dump(json_dic, f, indent=4)